In [37]:
import numpy as np
import pandas as pd
import sympy as sp
from matplotlib import pyplot as plt

In [56]:
#-------------------------------Find k----------------------------------------------
E = 6e04
ko = 1e-04
R = 8.314
To = 1140
def k(T):
    k = ko*np.exp(-E/(R*T))
    return k
k = k(To)

In [57]:
Fs = 1.26
V = 1
#---------------------Steady state outlet concentrations-----------------------------
Cas = 82.45
Cbs = 247.36
Ccs = 352.76
#---------------------Steady state inlet concentrations-----------------------------
Caos = 258.83
Cbos = 776.5
Ccos = 0

Kp = 1/12
tau = 10
ksi = 3

Ccv = 0.09662
deltaP_cv = 2
rho = 8.81609
SG = rho/1000
xs = 1/12*9

## Differential equations
The differential equations used to calculate the transfer functions are given below. 

Note that A is $N_2$, B is $H_2$, and C is $NH_3$

$$
\begin{bmatrix}
\frac{dC_A'}{dt} \\
\frac{dC_B'}{dt} \\
\frac{dC_C'}{dt}
\end{bmatrix}
=
\begin{bmatrix}
(-\frac{Fs}{V} - kC_{Bs}^3) & (-3kC_{As} C_{Bs}^2) & 0 \\
(-3kC_{Bs}^3) & (-\frac{Fs}{V} - 9kC_{As}C_{Bs}^2) & 0 \\
(2kC_{Bs}^3) & (6kC_{as}C_{Bs}^2) & (-\frac{Fs}{V})
\end{bmatrix}
\cdot
\begin{bmatrix}
C_A' \\
C_B' \\
C_C'
\end{bmatrix}
+
\begin{bmatrix}
\frac{C_{Ao}-C_A}{V} \\
\frac{C_{Bo}-C_B}{V} \\
0
\end{bmatrix}
\cdot
\begin{bmatrix}
F' 
\end{bmatrix}
\begin{bmatrix}
\frac{Fs}{V} \\
0 \\
0
\end{bmatrix}
\cdot
\begin{bmatrix}
C_{Ao}' 
\end{bmatrix}
$$



In [58]:
#-------------------------------A Matrix----------------------------------------------
a11 = -Fs/V-k*Cbs**3
a12 = -3*k*Cas*Cbs**2
a13 = 0
a21 = -3*k*Cbs**3
a22 = -Fs/V-9*k*Cas*Cbs**2
a23 = 0
a31 = 2*k*Cbs**3
a32 = 6*k*Cas*Cbs**2
a33 = -Fs/V
#---------------------B Matrix (coefficients of MV)-----------------------------------
b1 = (Caos-Cas)/V
b2 = (Cbos-Cbs)/V
b3 = -Ccs/V
#---------------------E Matrix (coefficients of DV)-----------------------------------
e1 = Fs/V
e2 = 0
e3 = 0

## Finding Gp(s)

In [59]:
A = sp.Matrix([[a11,a12,a13],[a21,a22,a23],[a31,a32,a33]])    #Coefficients of Ca, Cb, Cc
B = sp.Matrix([[b1],[b2],[b3]])                               #Coefficients of MV
E = sp.Matrix([[e1],[e2],[e3]])                               #Coefficients of DV
C = sp.Matrix([[0,0,1]])                                      #Indicates controlled variable is Cc
I = sp.eye(A.shape[0])

In [60]:
Cc,F,s = sp.symbols('Cc,F,s')
Gp1 = C*(s*I - A).inv()*B
Gp1 = sp.nsimplify(Gp1[0,0]).simplify()
display(sp.Eq(Cc/F,Gp1))

Eq(Cc/F, 26457*(-269586130178937*s**2 - 679357048050910*s - 427994940272065)/(25*(808758390536811*s**3 + 11778044077993900*s**2 + 25828716614095900*s + 15463181029766200)))

The transfer function above relates the outlet concentration of Ammonia ($C_c$) to the flow rate ($F$).

We now need to find $\frac{F}{P_s}$ so that we can multiply $\frac{F}{P_s}$ by the transfer function $\frac{C_c}{F}$ (above) in order to get the overall transfer function $\frac{C_c}{P_s}$.


### Method
To do this we begin by finding x/Ps. That is, how does the stem position x change with the pressure supplied. 

In [61]:
d11 = 0
d12 = 1
d21 = -1/tau**2
d22 = -2*ksi/tau

f1 = 0
f2 = Kp/tau**2

In [62]:
A2 = sp.Matrix([[d11,d12],[d21,d22]])
B2 = sp.Matrix([[f1],[f2]])
C2 = sp.Matrix([[1,0]])
I2 = sp.eye(A2.shape[0])

In [63]:
x,Ps,s = sp.symbols('x,Ps,s')
Gp2 = C2*(s*I2 - A2).inv()*B2
Gp2 = sp.nsimplify(Gp2[0,0]).simplify()
display(sp.Eq(x/Ps,Gp2))

Eq(x/Ps, 1/(12*(100*s**2 + 60*s + 1)))

### The next step

Now that we have $\frac{x}{P_s}$, we can find $\frac{F}{P_s}$. We know that after linearizing $f(x)$, $F' = k_1 \cdot x'$ where $k_1$ is some constant.

Re-arranging, we get that $x' = \frac{F'}{k_1}$.

We can substitute that into the equation above and find that $\frac{F'}{P_s} = k_1 \cdot \frac{x}{P_s}$.

#### F
$$F = C_{cv} \cdot \sqrt{x} \cdot \sqrt{\frac{\Delta P_{cv}}{SG}}$$

After linearisation:
$$F = C_{cv} \cdot \sqrt{\frac{\Delta P_{cv}}{SG}} \cdot \left(\sqrt{x_s} + \frac{x'}{2\sqrt{x_s}}\right)$$

#### k1

After expressing in terms of deviation variables, we find that k1 is given by:

$$k_1 = \frac{C_{cv}}{2} \cdot \sqrt{\frac{\Delta P_{cv}}{x_s \cdot SG}}$$

Substituting in the numerical values we know, we can multiply $k_1$ by the transfer function $\frac{x}{P_s}$ to obtain a new transfer function $\frac{F}{P_s}$ shown below.

In [53]:
F,Ps,s = sp.symbols('F,Ps,s')
Gp3 = Gp2*(Ccv/2*(deltaP_cv/xs/SG)**0.5) 
display(sp.Eq(F/Ps,Gp3))

Eq(F/Ps, 0.0700167825648624/(100*s**2 + 60*s + 1))

In [54]:
Gp = Gp1*Gp3

Cc,Ps,s = sp.symbols('Cc,Ps,s')
display(sp.Eq(Cc/Ps,Gp))

Eq(Cc/Ps, 74.0973606527426*(-269586130178937*s**2 - 679357048050910*s - 427994940272065)/((100*s**2 + 60*s + 1)*(808758390536811*s**3 + 11778044077993900*s**2 + 25828716614095900*s + 15463181029766200)))

## Finding Gd(s)

All that remains to find Gd(s) is to switch out the B matrix with the E matrix defined before to find the contribution made by the disturbance variable.

In [69]:
Cc,C_Ao,s = sp.symbols('Cc,C_Ao,s')
Gd = C*(s*I - A).inv()*E
Gd = sp.nsimplify(Gd[0,0]).simplify()
display(sp.Eq(Cc/C_Ao,Gd))

Eq(Cc/C_Ao, 252*(109015022377282*s + 137358928195375)/(5*(808758390536811*s**3 + 11778044077993900*s**2 + 25828716614095900*s + 15463181029766200)))

In [70]:
# Hein add your code about the poles etc. 

# Task 2: Block Diagram

![Block Diagram](Block_Diagram.png)

In [13]:
def rates(var):
    #fill in function

SyntaxError: incomplete input (3727379525.py, line 2)

In [ ]:
def flow_valve(x):
    #fill in function

In [10]:
def ManiVar(t):
    return Pso if t<step_time else Pso + delta_Ps

In [ ]:
def diff(t,var,MV):
    #fill in function

In [7]:
def Euler(t,dt,var,MV):
    de_var = diff(t,var,MV)
    var = de_var*dt + var
    return var

In [8]:
def Euler_Integration(tspan,dt,ss_var):
    Values = []
    var = ss_var
    for i,t in enumerate(tspan):
        MV = ManiVar(t)
        var = Euler(t,dt,var,MV)
        Values.append(var)
    return Values